# 수집된 데이터에 대한 통계 분석
* 데이터 
> 2017년 12월 1일부터 2018년 1월 1일까지 32일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [31]:
import os
import pandas as pd 
from glob import glob
import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re
from operator import itemgetter
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from ckonlpy.tag import Twitter as ctwitter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from collections import namedtuple
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
%matplotlib inline

In [160]:
#site = 'Naver'
site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
useCollection = dh.Use_Collection(useDb, collection)

In [3]:
mecab = Mecab()
def tokenize_pos2(doc):
    return ['/'.join(t) for t in mecab.pos(doc)]

In [4]:
rawdata = pd.read_csv('./data/sentiment_data/raw_data_for_sentiment.txt',header=None, encoding='utf-8')
rawdata.head()
X = list(rawdata[0])
y = np.array(list(rawdata[1]), dtype=int)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model_mecab_log1 = Pipeline([
            ('vect', TfidfVectorizer(tokenizer=tokenize_pos2, ngram_range=(1,2))), 
            ('log', LogisticRegression()),
        ])

In [ ]:
%%time 
model_mecab_log1.fit(train_X, train_y)
print(classification_report(test_y, model_mecab_log1.predict(test_X)))

In [ ]:
pickle.dump(model_mecab_log1, open('./data/pre_data/classifier_log_classifier_ngram12_by_mecab.pickled','wb'))

In [6]:
model_mecab_log1 = pickle.load(open('./data/pre_data/classifier_log_classifier_ngram12_by_mecab.pickled','rb'))

In [36]:
model = doc2vec.Doc2Vec.load('./model/doc2vec_size2000_epoch20_by_mecab.model')
classifier = pickle.load(open('./data/pre_data/classifier_by_mecab_from_doc2vec_size2000_epoch20.pickled','rb'))

* 주의 
> gensim의 doc2vec 모델의 infer_vector 함수를 이용해 벡터를 생성하면, 같은 단어열(문서)에 대해서도 여러번 호출하면 각기 다른 벡터가 나온다  
> 이는 문서 벡터 추론 과정에서 랜덤하게 초기화하는 과정이 있기 때문인데, 문제는 결과로 나오는 벡터의 차이가 꽤 크다.  

In [37]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

In [187]:
def Cls(text, model, num):
    mecab = Mecab()
    def tokenize_pos2(doc):
        return ['/'.join(t) for t in mecab.pos(doc)]
    TaggedDocument = namedtuple('TaggedDocument', 'words tags')
    x2 = TaggedDocument(tokenize_pos2(text), 0.5)
    out = list()
    for idx in range(num):
        x = model.infer_vector(x2.words)
        cl = classifier.predict(x.reshape(1,-1))
        out.append(cl)
    out = list(itertools.chain.from_iterable(out))
    cls2 = int(round(sum(out)/len(out)))
    return cls2

In [196]:
keywordsDict = pickle.load(open('./data/pre_data/keywords_daum.pickled','rb'))
#keywordsDict = pickle.load(open('./data/pre_data/keywords_Naver.pickled','rb'))

In [198]:
i=0
collection = useCollection.find({'site':site})
for data in collection:
    idis = data['_id']._ObjectId__id.hex()
    
    text = data['title']+'. '+data['mainText']
    print (data['title'], Cls(text, model, 50))
    commentCollection = dh.Use_Collection(useDb, 'comments')
    comments = commentCollection.find({'site':site, 'category':data['category'], 'date':data['date'], 'rank':data['rank'] })
    commEstimation = list()
    for comment in comments:
        comm = comment['comments']
        commEstimation.append(Cls(comm, model, 30))
    cls = sum(commEstimation)/ len(commEstimation)
    print (keywordsDict[idis])
    print ('댓글수 : {}, 댓글 판단 : {:0.2f}'.format(comments.count(), cls))
    if i==15:break
    i+=1

김국진·노홍철·한혜진 '한명회', 오늘(5일) 9회만에 종영 0
{'한명회', '특집', '이름', '녹화', '출연자', '방송'}
댓글수 : 120, 댓글 판단 : 0.53
'몸 만드는 게 우선' 류중일 감독, 캠프 명단에 신인 없다 0
{'신인', '류중일 감독', '캠프 명단', '오키나와 캠프', '선수들'}
댓글수 : 27, 댓글 판단 : 0.70
[어제TV]첫방 '저글러스' 男보스 女비서, 시대착오적 설정 '불편' 0
{'아내', '비서', '보스', '불륜', '설정', '무릎', '수습', '봉상무', '좌윤이'}
댓글수 : 125, 댓글 판단 : 0.50
[Oh! 무비] '기억의 밤' 장항준 감독은 어떻게 골리앗을 무찔렀나 1
{'장항준 감독', '시나리오', '기억의', '드라마', '영화', '스릴러'}
댓글수 : 39, 댓글 판단 : 0.59
[RE:TV]'섬총사' 강호동X정상훈, 80cm 대어 낚아 '낚시TV 따로없네' 0
{'농어', '강호동과 정상훈', '어청도', '총사', '낚시', '조세호'}
댓글수 : 23, 댓글 판단 : 0.74
'범죄도시'는 왜 불법 다운과의 전쟁을 선포했나 0
{'유포 자들', '처벌', '행위', '범죄 도시', '불법 유포', '영화'}
댓글수 : 81, 댓글 판단 : 0.58
'다둥이 아빠' 박지헌, 여섯째 출산 앞둔 아내에 "정말 고마워" 애틋 0
{'아내', '인스타그램', '사랑', '여섯째 출산', '박지헌'}
댓글수 : 609, 댓글 판단 : 0.60
[SC이슈] "무서운 팬덤 화력"..강다니엘,  뉴욕 타임스퀘어 전광판 장식 1
{'타임스퀘어 전광판', '팬들', '뉴욕 타임스퀘어', '아이돌', '광고', '강다니엘'}
댓글수 : 87, 댓글 판단 : 0.80
반찬 먹는 순서만 바꿔도 살이 빠지고 당뇨병이 예방된다? 0
{'식이섬유', '생선', '순서', '혈당', '단백질 탄수화물', '먹는', '식사', '육류'}
댓글수 : 484, 댓글 판단 : 0.54
"지금에 감사하자♡".